In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from openaq import OpenAQ
from datetime import datetime
from prophet import Prophet
import folium
import time
import os
from dotenv import load_dotenv

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [ ]:
#loading health data
health_data = pd.read_csv("Cases_data.csv")

True

In [3]:
#loading airquality data
load_dotenv(dotenv_path=".gitignore/.env")  # this loads .env file

API_KEY = os.getenv("API_KEY")

headers = {
    "X-API-Key": API_KEY
}
headers = {
    'accept': 'application/json',
    'X-API-KEY': os.getenv("API_KEY")
}

params = {
    'date_to': '2023-12-31T23:59:59Z',
    'date_from': '2021-01-01T00:00:00Z',
    'limit': 1000,
    'page': 1,
}


# To store all results
all_data = []

while True:
    response = requests.get('https://api.openaq.org/v3/sensors/23534/days/monthly', params=params, headers=headers)

    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        break

    data = response.json()
    results = data.get("results", [])

    if not results:
        break

    all_data.extend(results)
    print(f"Fetched page {params['page']}, total records: {len(all_data)}")

    # Check if there is a next page
    meta = data.get("meta", {})
    if not meta.get("found") or (params['page'] * params['limit'] >= meta.get("found", 0)):
        break

    params["page"] += 1

# Convert to DataFrame
df = pd.DataFrame(all_data)

# Show a sample
print(df.head())

# Save to CSV
df.to_csv("delhi_pm25_2021_2023.csv", index=False)
print(f"Saved {len(df)} records to delhi_pm25_2021_2023.csv")

#data_cleaning
df['date_from'] = df['period'].apply(
    lambda d: pd.to_datetime(d['datetimeFrom']['local']).strftime("%d-%m-%y")
)
df['date_to'] = df['period'].apply(
    lambda d: pd.to_datetime(d['datetimeTo']['local']).strftime("%d-%m-%y")
)

airQuality_data = pd.read_csv("delhi_pm25_2021_2023.csv")

Fetched page 1, total records: 36
   value             flagInfo  \
0  211.0  {'hasFlags': False}   
1  154.0  {'hasFlags': False}   
2   88.6  {'hasFlags': False}   
3   72.6  {'hasFlags': False}   
4   48.7  {'hasFlags': False}   

                                           parameter  \
0  {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'd...   
1  {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'd...   
2  {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'd...   
3  {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'd...   
4  {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'd...   

                                              period coordinates  \
0  {'label': '1 month', 'interval': '1 month', 'd...        None   
1  {'label': '1 month', 'interval': '1 month', 'd...        None   
2  {'label': '1 month', 'interval': '1 month', 'd...        None   
3  {'label': '1 month', 'interval': '1 month', 'd...        None   
4  {'label': '1 month', 'interval': '1 month', 'd...        None   

                     

In [ ]:
#data_cleaning of delhi_pm25 file
df['date_from'] = df['period'].apply(
    lambda d: pd.to_datetime(d['datetimeFrom']['local']).strftime("%d-%m-%y")
)
df['date_to'] = df['period'].apply(
    lambda d: pd.to_datetime(d['datetimeTo']['local']).strftime("%d-%m-%y")
)

airQuality_data = pd.read_csv("delhi_pm25_2021_2023.csv")

In [4]:
#data cleaning of cases data
df = pd.read_csv("Cases_data.csv")

df["date"] = pd.to_datetime(
    df["Month"].astype(str) + " " + df["Year"].astype(str) + " 01",
    format="%B %Y %d",   # Use %b instead of %B if months are in short form (Jan, Feb)
    errors="coerce"      # In case there are invalid values
)

# Convert to dd-mm-yy
df["date"] = df["date"].dt.strftime("%d-%m-%y")
df.to_csv("Cases_data.csv", index=False)
print(df["date"])

0     01-01-21
1     01-02-21
2     01-03-21
3     01-04-21
4     01-05-21
5     01-06-21
6     01-07-21
7     01-08-21
8     01-09-21
9     01-10-21
10    01-11-21
11    01-12-21
12    01-01-22
13    01-02-22
14    01-03-22
15    01-04-22
16    01-05-22
17    01-06-22
18    01-07-22
19    01-08-22
20    01-09-22
21    01-10-22
22    01-11-22
23    01-12-22
24    01-01-23
25    01-02-23
26    01-03-23
27    01-04-23
28    01-05-23
29    01-06-23
30    01-07-23
31    01-08-23
32    01-09-23
33    01-10-23
34    01-11-23
Name: date, dtype: object
